<a href="https://colab.research.google.com/github/lszeremeta/road-sign-recognition/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hyperopt

In [20]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_tree/road-sign-recognition'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_tree/road-sign-recognition


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

def predict(trained_model, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = trained_model.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Flatten(),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(num_classes, activation='softmax'),
                     ])

In [16]:
model = get_cnn_v5(input_shape, num_classes)
trained_model = train_model(model, X_train, y_train)

predict(trained_model, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.6118 - accuracy: 0.2838 - val_loss: 1.1475 - val_accuracy: 0.6091
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 0.9806 - accuracy: 0.6813 - val_loss: 0.3226 - val_accuracy: 0.9094
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.4539 - accuracy: 0.8548 - val_loss: 0.1057 - val_accuracy: 0.9693
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2830 - accuracy: 0.9135 - val_loss: 0.0761 - val_accuracy: 0.9762
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2137 - accuracy: 0.9360 - val_loss: 0.0365 - val_accuracy: 0.9897


0.9709750566893424

In [0]:
def get_model(params):
  return Sequential([
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_one']),

                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_two']),

                     Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_three']),

                     Flatten(),

                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_one']),

                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_two']),

                     Dense(num_classes, activation='softmax'),
                     ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [36]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.31425400667189507, 'dropout_cnn_block_three': 0.42684295200076183, 'dropout_cnn_block_two': 0.38054210047950954, 'dropout_dense_block_one': 0.6798906797108684, 'dropout_dense_block_two': 0.40170542603808956}
accuracy=0.9396825432777405
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3484793595271037, 'dropout_cnn_block_three': 0.37701958384519857, 'dropout_cnn_block_two': 0.3912824594951585, 'dropout_dense_block_one': 0.5384070122772382, 'dropout_dense_block_two': 0.4335506391173245}
accuracy=0.9580498933792114
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.41320418624566657, 'dropout_cnn_block_three': 0.48027944311946863, 'dropout_cnn_block_two': 0.3467917834067491, 'dropout_dense_block_one': 0.3585385646247054, 'dropout_dense_block_two': 0.3145447199353209}
accuracy=0.8977324366569519
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4779834338587711, 'dropout_cnn_block_three': 0.4675599969420674, 'dropout_cnn_block_two': 0.319476872

In [0]:
def get_cnn_v6(input_shape, num_classes):
  return Sequential([
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Flatten(),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(num_classes, activation='softmax'),
                     ])

In [38]:
model = get_cnn_v6(input_shape, num_classes)
trained_model = train_model(model, X_train, y_train)

predict(trained_model, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 10s 36ms/step - loss: 2.6889 - accuracy: 0.2504 - val_loss: 0.8557 - val_accuracy: 0.7267
Epoch 2/5
272/272 [==============================] - 6s 21ms/step - loss: 0.6602 - accuracy: 0.7848 - val_loss: 0.2004 - val_accuracy: 0.9306
Epoch 3/5
272/272 [==============================] - 6s 20ms/step - loss: 0.2857 - accuracy: 0.9124 - val_loss: 0.0758 - val_accuracy: 0.9758
Epoch 4/5
272/272 [==============================] - 6s 21ms/step - loss: 0.1649 - accuracy: 0.9518 - val_loss: 0.0388 - val_accuracy: 0.9895
Epoch 5/5
272/272 [==============================] - 6s 21ms/step - loss: 0.1229 - accuracy: 0.9660 - val_loss: 0.0385 - val_accuracy: 0.9887


0.9723356009070295

In [39]:
trained_model.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 4ms/step - loss: 0.0878 - accuracy: 0.9723


0.9723355770111084

In [0]:
def get_model2(params):
  return Sequential([
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_one']),

                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_two']),

                     Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                     Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_three']),

                     Flatten(),

                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_one']),

                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_two']),

                     Dense(num_classes, activation='softmax'),
                     ])

In [0]:
def func_obj2(params):
  model = get_model2(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [42]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj2,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 130.0, 'dropout_cnn_block_one': 0.32046540413371005, 'dropout_cnn_block_three': 0.4620893425769784, 'dropout_cnn_block_two': 0.4666188200607172, 'dropout_dense_block_one': 0.6271111682259503, 'dropout_dense_block_two': 0.4797406433016963}
accuracy=0.96417236328125
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.4663353925633011, 'dropout_cnn_block_three': 0.33424199954863343, 'dropout_cnn_block_two': 0.38432001009194594, 'dropout_dense_block_one': 0.5484583692330355, 'dropout_dense_block_two': 0.4409265472896334}
accuracy=0.9752834439277649
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.42823865182182497, 'dropout_cnn_block_three': 0.44474065827546905, 'dropout_cnn_block_two': 0.3013935986524188, 'dropout_dense_block_one': 0.5228674274206675, 'dropout_dense_block_two': 0.6578319615216565}
accuracy=0.9634920358657837
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.48898282263250215, 'dropout_cnn_block_three': 0.39781475766836644, 'dropout_cnn_block_two': 0.44149709474